In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

**what a derivative is and what information this tool provides:**

The derivative is the instantaneous rate of change of a function with respect to one of its variables. This is equivalent to finding the slope of the tangent line to the function at a point.

In [ ]:
#This is a scalar function: get single scalar x and returns a single scalar
#It is the equation of a parabola quatratic 
def f(x):
    return 3*x**2 - 4*x + 5

In [ ]:
print(f(3.0))

Plot the function to understand its shape

In [ ]:
#let create a list of scalars
xs = np.arange (-5,5,0.25)
#xs
ys = f(xs)
ys

In [ ]:
print (ys)

In [ ]:
plt.plot(xs,ys)

Question: what is the derivative of this functions at any single input point x ? What is the derivative at different points x of this function? 
What the derivative is measuring and what it's telling about the function?

If we slight change the at some point a small number h how does the function respond.
What is the sensitivity of the slope at that point? it is going down and up and by how much?

L = lim (h->0) (f(a+h) - f(a))/h

In [ ]:

# 3*x**2 - 4*x + 5 = 6X - 4 => 6*3 - 4 = 18 - 4 = 14
h = 0.000001
x = 2/3
(f(x+h) - f(x))/h

In [ ]:
#function of three scalars
a = 2.0
b = -3.0
c = 10.0
d = a*b + c
print(d)

In [ ]:
#get a very small value of h
h = 0.0001
#inputs
a = 2.0
b = -3.0
c = 10.0

#find the derivative of d at all that points
d1 = a*b + c
a += h
d2 = a*b + c #by calculus the diffentiation of a*b + c - it is just b what is negative number

print('d1', d1)
print('d2', d2)
#d1 - d2 it is how much the function increases (d2 compare to d1) and this is normalise by h (the tiny amount bump in in d2) to get the slope
print('slope', (d2 - d1)/h)
#d2 will be slightly less then d1 as the scalar b is negative, if even we add a positive amount to a


In [ ]:
#find the derivative of d at all that points
# in respect of b
d1 = a*b + c
b += h #it is adding more to b
d2 = a*b + c #by calculus the diffentiation in respect to a of a*b + c - it is b with is 2

print('d1', d1)
print('d2', d2)
#d1 - d2 it is how much the function increases (d2 compare to d1) and this is normalise by h (the tiny amount bump in in d2) to get the slope
print('slope', (d2 - d1)/h)
#d2 as a is positive it means to add more to b

In [ ]:
#find the derivative of d at all that points
#in respect of c
d1 = a*b + c
c += h #it is adding more to d (c will effect for the same amount that we add to c, then 1)
d2 = a*b + c #by calculus the diffentiation of a*b + c - it is just b what is negative number

print('d1', d1)
print('d2', d2)
#d1 - d2 it is how much the function increases (d2 compare to d1) and this is normalise by h (the tiny amount bump in in d2) to get the slope
print('slope', (d2 - d1)/h)


In [ ]:
class Value:
    #childre default empty tuple (it introduces for efficiency)
    def __init__(self, data, _children=(), _op='', label='', grad=0.0):
        self.data = data
        self.grad = grad #in initialisation the data does not impact the loss function
        self._prev = set(_children)
        self._op = _op
        self.label = label 
        
    def __repr__(self):
        return f"Value(data={self.data})"
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other),'+')
        return out
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other),'*')
        return out
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        return out

a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
#a + b #python does not know how to add two objects together
#value object with data = 2
#a.__add__(b)
#d = a*b + c
#(a.__mul__(b)).__add__(c)
#d
f = Value(-2.0, label = 'f')
L = d*f; L.label = 'L'
L
#d._prev
#d._op

In [ ]:
#visualisation the expression

from graphviz import Digraph

def trace(root):
    #builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    #call function build
    build(root)   
    #return nodes and edges sets 
    return nodes, edges
        
def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir':'LR'})
    
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        #for any value in the graph, create a rectangular ('record') node for it
        dot.node(name = uid, label = "{%s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')  
        if n._op:
            #if this value is a result of some operation, create an op node for it
            dot.node(name = uid + n._op, label = n._op) 
            #and connect this node to it
            dot.edge(uid+n._op, uid)
    for n1, n2 in edges:
        #connect n1 to the op node of n2
        dot.edge(str(id(n1)),str(id(n2))+n2._op)
    return dot


In [ ]:
draw_dot(L)

In [ ]:
#how L is sensitive to c
#we know how the impact c impacts d and we know how d impacts L
L = d * f #So it will be d so what is the dd/dc
d = c + e # it will be 1
#(f(x+h) - f(x)) / x
#((c+h  + e - (c + e))/ h
#(c + h  + e - c - e)/ h
#h/h = 1
#dd/dc ? = 1.0 
#and by symmetry
#dd / de = 1.0
#chain rule  to get until L (function composition) - intuitive explanation - product of two functions
#dz/dx = (dz/dy)*(dy/dx)
#find the local gradient

#WANT
#dL / dc = (dl/ dd) * (dd / dc) = -2.0 * 1.0
#dL / da = (dL / de) * (de / da) = -2.0 * -3.0
#dL / dbb = (dL / de) * (de / db) = -2.0 * 2.0


#KNOW
#dL / de = -2.0
#e = a * b
#de / da ? b
#de / db ? a

#WE TAKE NODE BY NODE and locally apply the chain rules and we have the pointer to child nodes of these operations


In [ ]:
c.grad = -2.0 * 1.0
e.grad = -2.0 * 1.0
a.grad = -2.0 * -3.0
b.grad = -2.0 * 2.0
f.grad = 4.0
d.grad = -2.0
L.grad = 1.0

In [ ]:
#dL/dc = ?f 
#(f(x+h)-f(x))/h
#((d+h)*f - d*f)/h
#(d*f + h*f - d*f) /h
#(h*f)/h
#f


In [ ]:
a.data += 0.01 * a.grad
b.data += 0.01 * b.grad
c.data += 0.01 * c.grad
f.data += 0.01 * f.grad

e = a * b
d = e + c
L = d * f

print (L.data)
#draw_dot(L)


We compute the derivate in respected to L and all variables respect to the Loss (L) function

How does the weight impact the LOSS function? We represent the derivative of the output in respect of L

In [ ]:
def lol():
    #change amount in function and check how it behave
    h = 0.0001

    a = Value(2.0, label='a')
    b = Value(-3.0, label='b')
    c = Value(10.0, label='c')
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2.0, label = 'f')
    L = d*f; L.label = 'L'
    L1 = L.data #copy the value of the object

    a = Value(2.0, label='a')
    a.data += h
    b = Value(-3.0, label='b')
    c = Value(10.0, label='c')
    #c.data += h
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    #d.data += h
    f = Value(-2.0, label = 'f')
    L = d*f; L.label = 'L'
    L2 = L.data
    print((L2 -L1)/h)

lol()

In [ ]:
#basically neuron, we have synapse and this synapse has weights
plt.plot(np.arange(-5,5,0.2), np.tanh(np.arange(-5,5,0.2))); plt.grid();
#squash function

In [ ]:
#inputs x1, x2 two dimension neuron
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
#weights w1, w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
#bias of the neuron
b = Value(6.881373, label = 'b')
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
#x1*w1 + x2*w2 
x1w1x2w2= x1w1 + x2w2; x1w1x2w2.label = 'x1w1x2w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o' #hyperbolic functons
draw_dot(o)